# LPR Assignment using OpenCV and Tesseract

Use Optical Character Recognition (OCR) to extract license plate number, aka. vehicle registration number (VRN)
The actual VRN is coded in the file name and also represented in the corresponding meta data file (XML file)

In [1]:
import sys
# sys.path.insert(0, "/opt/opencv-4.4.0/build/python_loader")
import cv2
print(cv2.__version__)

4.4.0


In [7]:
import sys, os
jp = os.path.join
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

##DATADIR = '/data/classes/MSA8650/LPR_project_data'
DATADIR = 'data_redatcted'

## OPENCV = '/opt/local/lib/python3.6/site-packages/cv2/python-3.6'
# OPENCV = '/opt/opencv-4.4.0/build/lib/python3'
# if OPENCV not in sys.path:
#     sys.path.insert(0, OPENCV)
# import cv2
# print(cv2.__version__)

import pytesseract
print(pytesseract.get_tesseract_version())

4.1.1


# Meta Info
Read the XML files and create a table with all the meta information of the samples.

In [ ]:
xmls_list = [x for x in filter(lambda s: s.endswith('.xml'), map(lambda s: s.strip(), os.popen('ls %s/xml' % DATADIR).readlines()))]
context_list = [x for x in filter(lambda s: s.endswith('.jpg'), map(lambda s: s.strip(), os.popen('ls %s/context' % DATADIR).readlines()))]
ir_list = [x for x in filter(lambda s: s.endswith('.jpg'), map(lambda s: s.strip(), os.popen('ls %s/ir_patch' % DATADIR).readlines()))]
xmls_list[:3], context_list[:3], ir_list[:3]

In [ ]:
lpdf = pd.DataFrame(index=map(lambda s: s.split('_')[0], xmls_list), data={'xml': xmls_list}) \
        .join( pd.DataFrame(index=map(lambda s: s.split('_')[0], context_list), data={'context': context_list}), how='outer' ) \
        .join( pd.DataFrame(index=map(lambda s: s.split('_')[0], ir_list), data={'ir_patch': ir_list}), how='outer' ) \
        .dropna().copy()
print(lpdf.shape)
display(lpdf.head(5))

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
meta_df.IRImagePatch.

In [ ]:
meta_df = pd.DataFrame()
for i, r in lpdf.iterrows():
    tree = ET.parse(jp(DATADIR, 'xml', r.xml))
    el_dict = {}
    for x in tree.iter():
        el_dict[x.tag] = [x.text]
    el_dict
    r_df = pd.DataFrame(el_dict) ##, index=[r.xml])
    r_df.index = [r.xml]
    meta_df = pd.concat([meta_df, r_df])
for c in ['IRImagePatch', 'ContextImage']:
    print(f'fix {c}')
    meta_df[c] = meta_df[c].map(lambda p: p.replace(r'data_redatcted/', ''))
print(meta_df.shape)
meta_df.head(5)

In [ ]:
meta_df.reset_index() \
    .rename({'index': 'XMLFile'}, axis=1) \
    .to_csv(jp(DATADIR, 'meta_data.csv'), index=None)

In [ ]:
! ls -l /data/classes/MSA8650/LPR_project_data/


In [ ]:
num_cols = ['ANPRImageHeight', 'ANPRImageWidth', 'PlateHeight', 'PlateWidth', 'PlateXCoord', 'PlateYCoord', 
            'TagConfidence', 'VRNConfidence', 'Velocity', 'VelocityError', 'XCoord', 'YCoord']
def string2float(x):
    return np.NAN if x=='None' else float(x)
    
for col in num_cols:
    meta_df[col] = meta_df[col].map(string2float)
meta_df.head(10)
meta_df.dtypes

In [ ]:
lpdf2 = lpdf.set_index('xml').join(meta_df).reset_index()
lpdf2.head(5)

In [ ]:
lpdf2[['PlateXCoord', 'PlateYCoord', 'PlateWidth', 'PlateHeight']].describe()

In [ ]:
lpdf2.describe()

In [ ]:
row = lpdf2.iloc[17]

gray = cv2.cvtColor(cv2.imread(jp(DATADIR, 'context', row.context)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
print gray.shape
plt.figure(figsize=(12.80, 10.24))
plt.imshow(gray, cmap='gray')
plt.show()
plt.close()
ir = cv2.cvtColor(cv2.imread(jp(DATADIR, 'ir_patch', row.ir_patch)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
plt.imshow(ir, cmap='gray')
plt.show()
plt.close()

# Adjusting position of IR-patch in context image

The context image size is 800x600 pixels, the meta data says 1280x1024. These two dimensions don't have the same aspect ratio. I suspect there is a scaling factor as well as some offset required to map the context image with the IR-patch.

The coordinates of the IR-patch are given as X, Y, Width, and Height. In order to determine whether the X, Y coordinates are in the center of the patch, or the upper-left corner I plotted both in green and red, respectively.

With the code below I tried some hypothesis of the scaling. 

In [ ]:
S = 600.0/1024
W = 1280*S
F = 800.0/1280.0
H = 1024*F
800.0/600.0, 1280.0/1024.0, S, W, F, H

In [ ]:
600*1.25

In [ ]:

for k in [230, 478, 1234, 97, 991]:
    row = lpdf2.iloc[k]
    gray = cv2.cvtColor(cv2.imread(jp(DATADIR, 'context', row.context)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
    ir = cv2.cvtColor(cv2.imread(jp(DATADIR, 'ir_patch', row.ir_patch)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
    plt.imshow(ir, cmap='gray')
    plt.figure(figsize=(8, 8))
    W =1600
    H = int(0.75*W)
    context2 = cv2.cvtColor(cv2.resize(gray, (W, H)), cv2.COLOR_GRAY2RGB)
    Sx = 1.0 ## 800.0/1280.0
    Sy = Sx ### 600.0/1024.0
    #Oy = H - int(H)
    x1 = int(Sx * (row.PlateXCoord - row.PlateWidth/2))
    y1 = int(Sy * (row.PlateYCoord - row.PlateHeight/2)) - int(Oy)
    x2 = int(Sx * (row.PlateXCoord + row.PlateWidth/2))
    y2 = int(Sy * (row.PlateYCoord + row.PlateHeight/2)) - int(Oy)
    x1_ = int(Sx * (row.PlateXCoord ))
    y1_ = int(Sy * (row.PlateYCoord )) - int(Oy)
    x2_ = int(Sx * (row.PlateXCoord + row.PlateWidth))
    y2_ = int(Sy * (row.PlateYCoord + row.PlateHeight)) - int(Oy)

    cv2.rectangle(context2, (x1, y1), (x2, y2), (0,255,0), 2)
    cv2.rectangle(context2, (x1_, y1_), (x2_, y2_), (255,0,0), 2)
    plt.imshow(context2)
    plt.show()
    plt.close()

# Haar Cascade

https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html

Object Detection using Haar feature-based cascade classifiers is a popular object detection method proposed. Commonly used for face-detection. There's a pre-trained model for Russian license plates. Though, doesn't look too promising

In [ ]:
!ls /opt/opencv-3.2.0/data/haarcascades/

In [ ]:
lpr_cascade = cv2.CascadeClassifier('/opt/opencv-3.2.0/data/haarcascades/haarcascade_licence_plate_rus_16stages.xml')
lpr_cascade

In [ ]:
for k in [230, 478, 1234, 97, 991]:
    row = lpdf2.iloc[k]
    gray = cv2.cvtColor(cv2.imread(jp(DATADIR, 'context', row.context)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
    plates = lpr_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in plates:
        print x, y
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,255),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        #     eyes = eye_cascade.detectMultiScale(roi_gray)
        #     for (ex,ey,ew,eh) in eyes:
        #         cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

    plt.imshow(img)
    plt.show()
    plt.close()

# Computing Mean and Variance from all Context Images
This method may help to find the region of interest (ROI) and remove any static background.

In [ ]:
gray_lst = []
for i, row in lpdf2.iterrows():
    gray = cv2.cvtColor(cv2.imread(jp(DATADIR, 'context', row.context)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
    gray_lst.append(gray)
print len(gray_lst)


In [ ]:
grays = np.array(gray_lst)
grays.shape

In [ ]:
plt.imshow(grays.mean(axis=0), cmap='gray')

And the variance ...

In [ ]:
plt.imshow(grays.var(axis=0), cmap='gray')

In [ ]:
img_variance = grays.var(axis=0)
var_max = img_variance.max()
plt.imshow(1.0*(img_variance>0.75*var_max), cmap='gray')

We may use this to crop images

# Vehicle Registration Number
I'm trying different image processing steps to prepare the image for OCR.

In [ ]:
import pytesseract
from PIL import Image
from Levenshtein import distance

In [ ]:
meta_df = pd.read_csv(jp(DATADIR, 'meta_data.csv'))
meta_df['VRN_length'] = meta_df.VRN.map(lambda s: len(s))
print(f'Meta-database: {meta_df.shape}')

In [ ]:
meta_df.columns

In [ ]:
imfile = jp(DATADIR, row['IRImagePatch'])
print(imfile)
ir = cv2.imread(jp(DATADIR, row['IRImagePatch']))
print(ir.shape)

In [ ]:
!ls -l /data/classes/MSA8650/LPR_project_data/ir_patch/15411_19700101131706973_PWL2646_1_ir_patch.jpg

In [ ]:
config = "-psm 7 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

row = meta_df.iloc[898]
img_file = jp(DATADIR, row['IRImagePatch'])
print(f"file: {img_file} {'exists' if os.path.exists(img_file) else 'does not exist'}")
ir = cv2.imread(jp(DATADIR, row['IRImagePatch']))
print(ir.shape)
ir = cv2.cvtColor(ir, cv2.COLOR_BGR2GRAY)
print(ir.shape)
plt.imshow(ir, cmap='gray');
plt.show()
plt.close()

In [ ]:
config = "-psm 7 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
res = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
print(res)

In [ ]:
np.array(meta_df.VRN.map(lambda s: len(s)))

In [ ]:
meta_df.groupby('VRN_length').agg({'VRN_length': 'count'})

In [ ]:
meta_df.columns

In [ ]:
def load_ir_image(row, quiet=True):
    img_file = jp(DATADIR, row['IRImagePatch'])
    assert os.path.exists(img_file), f"File '{img_file}' does not exist."
    ir = cv2.imread(img_file)
    if not quiet:
        print(ir.shape)
    ir = cv2.cvtColor(ir, cv2.COLOR_BGR2GRAY)
    if not quiet:
        print(ir.shape)
    return ir
          
    
def load_context_image(row, quiet=True):
    img_file = jp(DATADIR, row['ContextImage'])
    assert os.path.exists(img_file), f"File '{img_file}' does not exist."
    ir = cv2.imread(img_file)
    if not quiet:
        print(ir.shape)
    ir = cv2.cvtColor(ir, cv2.COLOR_BGR2RGB)
    if not quiet:
        print(ir.shape)
    return ir
          
    
def show_image(ir):
    plt.imshow(ir, cmap='gray', );
    plt.xticks([], [])
    plt.yticks([],[])
    plt.show()
    plt.close()

In [ ]:
for i, r in meta_df[meta_df.VRN_length==8].iterrows():
    print(r.VRN)
    ir = load_ir_image(r)
    show_image(ir)

In [ ]:
# N = (meta_df.VRN_length==7).sum()
# for i, r in meta_df[meta_df.VRN_length==4].iterrows():
#     print(r.VRN)
#     ir = load_context_image(r)
#     show_image(ir)

In [ ]:
N = (meta_df.VRN_length==4).sum()
for i, r in meta_df[meta_df.VRN_length==4].iterrows():
    print(r.VRN)
    ir = load_ir_image(r)
    show_image(ir)

In [ ]:
row = meta_df.iloc[1]
row

In [ ]:
ir = load_ir_image(row)
show_image(ir)

In [ ]:
from pytesseract import Output

In [ ]:
config = "-psm 7 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
print(pytesseract.image_to_boxes(ir, config=config))

In [ ]:
      
#gray = cv2.cvtColor(cv2.imread(jp(DATADIR, 'context', row.context)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
ir = cv2.cvtColor(cv2.imread(jp(DATADIR, 'ir_patch', row['IRImagePatch'])), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
plt.imshow(ir, cmap='gray');
plt.show()
plt.close()
# res = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
# print(f"VRN: {row.VRN} OCR: {res} Levenshtein distance: {distance(row.VRN, res)}")

# ir = cv2.resize(ir, (int(ir.shape[1]*1.5), ir.shape[0]), interpolation = cv2.INTER_AREA)
# plt.imshow(ir, cmap='gray');
# plt.show()
# plt.close()
# #ir = cv2.adaptiveThreshold(ir,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
# #            cv2.THRESH_BINARY,11,2)
# _ret, ir = cv2.threshold(ir,127,255,cv2.THRESH_BINARY)
# print type(ir)
# plt.imshow(ir, cmap='gray');
# plt.show()
# plt.close()
# ir = cv2.adaptiveThreshold(ir,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
#            cv2.THRESH_BINARY,11,2)
# plt.imshow(ir, cmap='gray');
# plt.show()
# plt.close()
# res = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
# print "VRN: %s OCR: %s Levenshtein distance: %d" % (row.VRN, res, distance(row.VRN, res))

Let's test out how well they do

In [ ]:
def process_ir(r):
    config = "-psm 7 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    ir = load_ir_image(r)
    res1 = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
    
    ir2 = cv2.resize(ir, (int(ir.shape[1]*1.5), ir.shape[0]), interpolation = cv2.INTER_AREA)
    res2 = pytesseract.image_to_string(Image.fromarray(ir2), config=config).strip()
    
    _ret, ir3 = cv2.threshold(ir,127,255,cv2.THRESH_BINARY)
    res3 = pytesseract.image_to_string(Image.fromarray(ir3), config=config).strip()
    
    ir4 = cv2.adaptiveThreshold(ir, 255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    res4 = pytesseract.image_to_string(Image.fromarray(ir4), config=config).strip()
    return (res1, res2, res3, res4)

In [ ]:
meta_df['OCR_results'] = meta_df.apply(process_ir, axis=1)

In [ ]:
config = "-psm 7 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

dist_plain = np.zeros(lpdf2.shape[0], dtype=float)
dist_scale = np.zeros(lpdf2.shape[0], dtype=float)
dist_bin = np.zeros(lpdf2.shape[0], dtype=float)
dist_bin_gauss = np.zeros(lpdf2.shape[0], dtype=float)


max_iter = 100
for i, row in lpdf2.iterrows():
    max_iter -= 1
    if max_iter<0: break
    #gray = cv2.cvtColor(cv2.imread(jp(DATADIR, 'context', row.context)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
    ir = cv2.cvtColor(cv2.imread(jp(DATADIR, 'ir_patch', row.ir_patch)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
    res = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
    dist_plain[i] = distance(row.VRN, res)
    
    ir = cv2.resize(ir, (int(ir.shape[1]*1.5), ir.shape[0]), interpolation = cv2.INTER_AREA)
    res = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
    dist_scale[i] = distance(row.VRN, res)

    _ret, ir = cv2.threshold(ir,127,255,cv2.THRESH_BINARY)
    res = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
    dist_bin[i] = distance(row.VRN, res)

    ir = cv2.adaptiveThreshold(ir, 255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    res = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
    dist_bin_gauss[i] = distance(row.VRN, res)

lpdf3 = lpdf2.copy()
lpdf3['dist_plain'] = dist_plain
lpdf3['dist_scale'] = dist_scale
lpdf3['dist_bin'] = dist_bin
lpdf3['dist_bin_gauss'] = dist_bin_gauss

In [ ]:
lpdf3.iloc[:100][['dist_plain', 'dist_scale', 'dist_bin', 'dist_bin_gauss']].describe()

In [ ]:
plt.hist(lpdf3.iloc[:100].dist_bin_gauss)

In [ ]:
row = lpdf2.iloc[278]
ir = cv2.cvtColor(cv2.imread(jp(DATADIR, 'ir_patch', row.ir_patch)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
plt.imshow(ir, cmap='gray')
res = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
print row.VRN, res, distance(row.VRN, res)

In [ ]:
import imutils
ir2 = imutils.rotate_bound(ir, angle=5)
plt.imshow(ir2, cmap='gray')
res = pytesseract.image_to_string(Image.fromarray(ir2), config=config).strip()
print row.VRN, res, distance(row.VRN, res)

In [ ]:
res = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
print row.VRN, res, distance(row.VRN, res)

In [ ]:
row = lpdf3[lpdf3.VehicleDirection=='U'].iloc[0]
ir = cv2.cvtColor(cv2.imread(jp(DATADIR, 'ir_patch', row.ir_patch)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
plt.imshow(ir, cmap='gray')
plt.show()
plt.close()

edges = cv2.Canny(ir,50,150,apertureSize = 3)
plt.imshow(edges, cmap='gray')
plt.show()
plt.close()
# This returns an array of r and theta values 
lines = cv2.HoughLines(edges,5,5*np.pi/180, 50)
if lines is not None:
    print len(lines)
else:
    print 0
# The below for loop runs till r and theta values  
# are in the range of the 2d array 
angles = np.array(map(lambda l: l[0][1]*180.0/np.pi, lines[:20]))
print angles
print angles.mean(), np.percentile(angles, 50)
# plt.hist(np.array(map(lambda l: l[0][1]*180.0/np.pi, lines[:2])), bins=20)
# plt.show()
# plt.close()

ir2 = imutils.rotate_bound(ir, angle=90-np.percentile(angles, 50))
plt.imshow(ir2, cmap='gray')
plt.show()
plt.close()
res = pytesseract.image_to_string(Image.fromarray(ir2), config=config).strip()
print row.VRN, res, distance(row.VRN, res)

In [ ]:
np.round(89.999995674289)

In [ ]:
print lines

In [ ]:
plt.hist(np.array(map(lambda l: l[0][1]*180.0/np.pi, lines)), bins=20)

In [ ]:
row = lpdf3[lpdf3.VehicleDirection=='U'].iloc[0]


In [ ]:
len(lines)

In [ ]:
row = lpdf2.iloc[1200]
ir = cv2.cvtColor(cv2.imread(jp(DATADIR, 'ir_patch', row.ir_patch)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
plt.imshow(ir, cmap='gray')
res = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
print row.VRN, res, distance(row.VRN, res)

In [ ]:
lpdf3.groupby('VehicleDirection')['VehicleDirection'].count()

In [ ]:
for i, row in lpdf3[lpdf3.VehicleDirection=='U'].iloc[:10].iterrows():
    ir = cv2.cvtColor(cv2.imread(jp(DATADIR, 'ir_patch', row['ir_patch'])), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
    plt.imshow(ir, cmap='gray')
    plt.show()
    plt.close()

In [ ]:
lpdf3.describe().T

# Process Pipeline

In [ ]:
def read_ir_patch(ir_patch):
    return cv2.cvtColor(cv2.imread(jp(DATADIR, 'ir_patch', ir_patch)), cv2.COLOR_BGR2GRAY)


def rotate(img):
    edges = cv2.Canny(img,50,150,apertureSize = 3)

    # This returns an array of r and theta values 
    lines = cv2.HoughLines(edges,5,5*np.pi/180, 50)
    if lines is not None:
        #print len(lines)
        thetas = np.array(map(lambda l: l[0][1]*180.0/np.pi, lines[:20]))
        th = np.round(np.percentile(angles, 50))
        img2 = imutils.rotate_bound(img, angle=90-th)
        return img2
    else:
        return img

    
def show(img):
    plt.imshow(img, cmap='gray')
    plt.show()
    plt.close()
    
    
def scale(img, Sx=1.0, Sy=1.0):
    img2 = cv2.resize(img, (int(img.shape[1]*Sx), int(img.shape[0]*Sy)), interpolation = cv2.INTER_AREA)
    return img2
    

def thresh(img, lo=127, hi=255):
    _ret, img2 = cv2.threshold(img, lo, hi, cv2.THRESH_BINARY)
    return img2


def threshgauss(img, lo=127, hi=255):
    _ret, img2 = cv2.threshold(img, lo, hi, cv2.THRESH_BINARY)
    img3 = cv2.adaptiveThreshold(img2, 255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    return img3


config = "-psm 7 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ"

lpdf4 = lpdf2.copy()

ls_dist_val = []
ls_dist_idx = []


max_iter = 99999999
for i, row in lpdf2.iterrows():
    max_iter -= 1
    if max_iter<0: break
    #gray = cv2.cvtColor(cv2.imread(jp(DATADIR, 'context', row.context)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
    ## ir = cv2.cvtColor(cv2.imread(jp(DATADIR, 'ir_patch', row.ir_patch)), cv2.COLOR_BGR2GRAY) ##  cv2.COLOR_BGR2RGB)
    ir = read_ir_patch(row.ir_patch)
    
    ir2 = scale(ir, Sx=2.0, Sy=2.0)
    ir3 = rotate(ir2)
    ir4 = threshgauss(ir3)
    
    res = pytesseract.image_to_string(Image.fromarray(ir4), config=config).strip()
    d = distance(row.VRN, res)
    ls_dist_val.append(d)
    ls_dist_idx.append(i)
    
    #     ir = cv2.resize(ir, (int(ir.shape[1]*1.5), ir.shape[0]), interpolation = cv2.INTER_AREA)
    #     res = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
    #     dist_scale[i] = distance(row.VRN, res)

    #     _ret, ir = cv2.threshold(ir,127,255,cv2.THRESH_BINARY)
    #     res = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
    #     dist_bin[i] = distance(row.VRN, res)

    #     ir = cv2.adaptiveThreshold(ir, 255,
    #                                cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    #     res = pytesseract.image_to_string(Image.fromarray(ir), config=config).strip()
    #     dist_bin_gauss[i] = distance(row.VRN, res)

dist_df = pd.DataFrame({'ls_dist': ls_dist_val}, index=ls_dist_idx)
print dist_df.shape
#dist_df.head()
dist_df.groupby('ls_dist')['ls_dist'].count()

In [ ]:
dist_df.groupby('ls_dist')['ls_dist'].count()

In [ ]:
angles = np.array(map(lambda l: l[0][0], lines))

In [ ]:
np.histogram(angles)

In [ ]:
ang2 = sorted(angles)
plt.plot(range(len(ang2)), ang2)